# 순환 신경망 기반 감성 분석 (영화 리뷰)

## 1. 데이터 준비
* 전처리된 한글 영화평 데이터 사용 (./data/Korean_movie_reviews_2016.csv)

In [ ]:
import pandas as pd
review_df = pd.read_csv('./data/Korean_movie_reviews_2016.csv')
review_df.head()

### 1-1. 입력-출력, 학습-테스트 데이터 분리

In [ ]:
# 입력 데이터와 정답 데이터 추출
input_list = list(review_df.review)
label_list = list(review_df.label)
len(input_list), len(label_list)

In [ ]:
# 입력 데이터를 문장별로 토큰 단위로 분리
tokenized_input_list = [review.strip().split() for review in input_list]
len(tokenized_input_list)

In [ ]:
# 범주별 데이터 수 차이 확인을 위해 label groupby 집계
review_df.groupby('label').size().reset_index(name='count')

In [ ]:
# 범주별 데이터 수로 막대그래프 그려보기
review_df.label.value_counts().plot(kind='bar')

In [ ]:
# 학습 데이터와 평가 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tokenized_input_list, label_list, test_size=0.1)
len(X_train), len(X_test), len(y_train), len(y_test)

### 1-2. 입력 데이터 Integer Encoding
* num_words = 사용한 단어 수 + 1 (0은 OOV에 할당)

#### [참고] 벡터에 사용할 단어수 구하기 (vocab_size)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 단어 수 제한없이 tokenizer 생성해보기
test_tokenizer = Tokenizer(num_words = max_features, oov_token=oov_token)

In [ ]:
# 등장 빈도수가 threshold 미만인 단어들이 이 데이터에서 얼만큼의 비중을 차지하는지 확인
threshold = 3
total_cnt = len(test_tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받아서, value가 threshold보다 작은 경우
# rare_cnt와 rare_freq를 update


#### 1-2-1. 단어 집합 및 index 생성

In [ ]:
# Integer Encoding을 위한 tokenizer 학습
from tensorflow.keras.preprocessing.text import Tokenizer
vocab_size = 25000
max_features = vocab_size + 1
tokenizer = Tokenizer(num_words = max_features)

# fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성.
tokenizer.fit_on_texts(X_train)
len(tokenizer.word_index)

#### 1-2-2. Integer Encoding

In [ ]:
# 학습 데이터 Integer Encoding
train_sequences = tokenizer.texts_to_sequences(X_train)
print(train_sequences[:5])

In [ ]:
# 테스트 데이터 Integer Encoding
test_sequences = tokenizer.texts_to_sequences(X_test)
print(test_sequences[:5])

#### 1-2-3. Integer Encoding이 길이가 0인 review 삭제

In [ ]:
# 각 샘플들의 길이를 확인해서 길이가 0인 샘플들의 인덱스를 받아오기
drop_train = [index for index, sequence  in enumerate(train_sequences) if len(sequence) == 0]
drop_test = [index for index, sequence  in enumerate(test_sequences) if len(sequence) == 0]
len(drop_train), len(drop_test)

In [ ]:
# 학습 데이터에서 삭제 -> drop_train에 없는 
new_train_sequence = [sequence for index, sequence in enumerate(train_sequences) if index not in drop_train]
new_y_train = [label for index, label in enumerate(y_train) if index not in drop_train]

In [ ]:
# 테스트 데이터에서 삭제
new_test_sequence = [sequence for index, sequence in enumerate(test_sequences) if index not in drop_test]
new_y_test = [label for index, label in enumerate(y_train) if index not in drop_train]
len(new_train_sequence), len(new_y_train)

In [ ]:
train_sequences = new_train_sequence
test_sequences = new_test_sequence
y_train = new_y_train
y_test = new_y_test

### 1-3. Padding

#### 1-3-1. 입력 벡터의 길이 정하기 (대부분의 문장이 잘리지 않는 크기)  

In [ ]:
# max_len 정하기 : 단어 길이의 histogram을 대부분을 포함하는 단어 길이 확인


In [ ]:
# 통계 정보 확인


In [ ]:
# 길이가 max_len 이하인 데이터의 비율 구하기


#### 1-3-2. 선택한 길이로 padding

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# max_len 설정
max_len = 45

# 학습 데이터 padding
X_train = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')

# 테스트 데이터 padding
X_test = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

print(X_train.shape, X_test.shape)
X_train[:5]

### 1-4. 정답 데이터 One-hot Encoding

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train.shape, y_test.shape

## 2. 모델 구축 (정의)

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
from tensorflow.keras.models import Sequential

input_units = max_features
embedding_dim = 32
rnn_units = 16
dense_units = 16
output_units = 2 

rnn_model = [
    Embedding(input_units, embedding_dim, input_length=max_len),
    SimpleRNN(rnn_units),
    Dense(dense_units, activation='tanh'),
    Dense(output_units, activation='softmax')
]

model = Sequential(rnn_model)
model.summary()

## 3. 모델 컴파일 (학습 설계)

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# 학습 설계 (옵티마이저, 손실함수)
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

## 4. 모델 학습

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# EarlyStopping, ModelCheckpoint callback 함수 설정
es = EarlyStopping(monitor='val_loss', mode='min', patience=3, verbose=1)
checkpoint_file = './model/best_model.h5'
mc = ModelCheckpoint(checkpoint_file, monitor='val_loss', mode='min', save_best_only=True)

In [ ]:
# 모델 학습
model.fit(X_train, y_train, epochs = 20, batch_size=128, validation_split=0.1, callbacks =[es, mc])

## 5. 모델 평가

In [ ]:
len(X_test), len(y_test)

In [ ]:
import numpy as np
# 저장된 모델 로딩하여, 테스트 데이터로 평가
model.load_weights(checkpoint_file)
_, test_acc = model.evaluate(X_test, y_test)
print(f'정확도 : {test_acc*100:.2f}%')

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# predict로 테스트 데이터의 예측값 구기기
preds = model.predict(X_test)

import numpy as np
#label = ['부정', '긍정']
pred_labels = [np.argmax(pred) for pred in preds]
pred_labels

In [ ]:
# sklearn의 classification_report()로 평가 결과 확인
from sklearn.metrics import classification_report
print(classification_report(new_y_test, pred_labels))

## 입력 문장 긍부정 판단

In [ ]:
# 입력된 문장의 긍부정 판단
review = "영화가 재미있다."

In [ ]:
# 긍부정 판단 함수
from konlpy.tag import Okt
t = Okt()
def sentiment_analysis(review):
    morphs = [word for word in t.morphs(review)] # 형태소 분석
    sequences = tokenizer.texts_to_sequences([morphs]) # Integer Encoding
    X = pad_sequences(sequences, maxlen=max_len) # Padding
    preds = model.predict(X)
    label = ['부정', '긍정']
    max_index = np.argmax(preds[0])
    result = label[max_index]
    return result, preds[0][max_index]

print(sentiment_analysis(review))

In [ ]:
# 함수 테스트
reviews = [
    '이 영화 개꿀잼 ㅋㅋㅋ',
    '하품만 나온다',
    '이 영화 핵노잼 ㅠㅠ',
    '이딴게 영화냐 ㅉㅉ',
    '와 개쩐다',
    '감독 뭐하는 놈이냐',
    '정말 세계관 최강자들의 영화다'
]

for review in reviews:
    result, prob = sentiment_analysis(review)
    print(f'{review} --> {result}({prob:.2f})\n')


In [ ]:
# 입력 받은 문장의 긍부정 판단

In [ ]:
import joblib
joblib.dump(tokenizer, "./model/movie_sa_tokeinzer.pkl")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=16, input_length=100))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# 감성 분석 학습 최종 모델
model.save("./model/movie_sa_model.keras")